In [1]:
!pip -q install --upgrade "llama-cpp-python[server]==0.2.84" google-adk litellm pandas matplotlib python-dateutil


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 MB 16.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 23.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. Thi

In [2]:
from huggingface_hub import hf_hub_download
import os

REPO_ID  = "TheBloke/DeepSeek-Coder-6.7B-Instruct-GGUF"
FILENAME = "deepseek-coder-6.7b-instruct.Q4_K_M.gguf"   # ~3.8 GB

model_path = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
os.environ["MODEL_PATH"] = model_path
print("MODEL_PATH =", os.environ["MODEL_PATH"])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


deepseek-coder-6.7b-instruct.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

MODEL_PATH = /root/.cache/huggingface/hub/models--TheBloke--DeepSeek-Coder-6.7B-Instruct-GGUF/snapshots/9e221e6b41cb1bf1c5d8f9718e81e3dc781f7557/deepseek-coder-6.7b-instruct.Q4_K_M.gguf


In [3]:
%%bash
# 3.A1 — start server in background and log to /content/server.log
pkill -f "llama_cpp.server.*--port 8000" || true

nohup python -m llama_cpp.server \
  --model "$MODEL_PATH" \
  --host 127.0.0.1 --port 8000 \
  --model_alias deepseek-local \
  --chat_format chatml \
  --n_ctx 4096 \
  > /content/server.log 2>&1 &

sleep 3
tail -n 50 /content/server.log


In [4]:
# 3.A2 — wait for server to be ready
import time, requests, subprocess

base = "http://127.0.0.1:8000"
for i in range(120):  # up to ~6 minutes on CPU
    try:
        r = requests.get(f"{base}/v1/models", timeout=2)
        if r.ok:
            print("Server is ready ✅")
            print(r.json())
            break
    except Exception:
        pass
    if i % 5 == 0:
        print(f"...waiting ({i*3}s)")
    time.sleep(3)
else:
    print("Server not ready; last 100 log lines:")
    subprocess.run(["tail","-n","100","/content/server.log"])


...waiting (0s)
Server is ready ✅
{'object': 'list', 'data': [{'id': 'deepseek-local', 'object': 'model', 'owned_by': 'me', 'permissions': []}]}


In [5]:
from dataclasses import dataclass, asdict
from datetime import datetime, timedelta
from typing import List, Dict, Any, Optional
import json, pandas as pd
from pathlib import Path

STORE = Path("/content/pm_tasks.json")

@dataclass
class Task:
    id: int
    title: str
    owner: str
    status: str          # "todo"|"doing"|"blocked"|"done"
    estimate_h: float    # original estimate
    spent_h: float       # time spent so far
    due: str             # ISO date
    tags: List[str]

def _load() -> List[Dict[str, Any]]:
    if STORE.exists():
        return json.loads(STORE.read_text())
    return []

def _save(rows: List[Dict[str, Any]]):
    STORE.write_text(json.dumps(rows, indent=2))

def seed_sample():
    if STORE.exists():
        return
    today = datetime.now().date()
    rows = [
        asdict(Task(1,"Auth flow","Aisha","doing",8,3,(today+timedelta(days=3)).isoformat(),["backend"])),
        asdict(Task(2,"Landing page","Miguel","todo",6,0,(today+timedelta(days=2)).isoformat(),["frontend"])),
        asdict(Task(3,"Billing webhook","Aisha","blocked",10,2,(today+timedelta(days=5)).isoformat(),["backend","payments"])),
        asdict(Task(4,"Onboarding email","Nora","done",3,3,(today+timedelta(days=1)).isoformat(),["growth"])),
        asdict(Task(5,"DB backup setup","Ravi","doing",5,1,(today+timedelta(days=4)).isoformat(),["infra"])),
    ]
    _save(rows)

def list_tasks(owner: Optional[str]=None, status: Optional[str]=None) -> Dict[str, Any]:
    rows = _load()
    if owner: rows = [r for r in rows if r["owner"].lower()==owner.lower()]
    if status: rows = [r for r in rows if r["status"].lower()==status.lower()]
    return {"tasks": rows}

def create_task(title:str, owner:str, estimate_h:float, due:str, tags:List[str]) -> Dict[str, Any]:
    rows = _load()
    new_id = (max([r["id"] for r in rows]) + 1) if rows else 1
    t = asdict(Task(new_id, title, owner, "todo", float(estimate_h), 0.0, due, tags))
    rows.append(t); _save(rows)
    return {"created": t}

def update_status(task_id:int, status:str, spent_delta:float=0.0) -> Dict[str, Any]:
    rows = _load(); found = None
    for r in rows:
        if r["id"]==task_id:
            r["status"] = status
            r["spent_h"] = max(0.0, r["spent_h"] + float(spent_delta))
            found = r; break
    if found is None: return {"error":"not_found"}
    _save(rows)
    return {"updated": found}

def compute_summary() -> Dict[str, Any]:
    rows = _load()
    df = pd.DataFrame(rows)
    if df.empty:
        return {"counts":{}, "burndown_points":[], "eta_days": None}
    counts = df["status"].value_counts().to_dict()
    total_est = df["estimate_h"].sum()
    total_spent = df["spent_h"].sum()
    remaining = max(0.0, total_est - total_spent)
    # naive ETA: remaining / avg_daily_throughput (assume 6h per dev-day)
    eta_days = round(remaining / 6.0, 1)
    # simple burndown (fake historic points): remaining declines linearly over 7 days
    pts = []
    for d in range(7, -1, -1):
        pts.append({"day": (datetime.now().date()-timedelta(days=d)).isoformat(),
                    "remaining_h": max(0.0, remaining + (d* (total_est/7.0 - total_spent/7.0)))})
    return {"counts": counts, "remaining_h": remaining, "eta_days": eta_days, "burndown_points": pts}

seed_sample()
print("Seeded:", len(_load()), "tasks")


Seeded: 5 tasks


In [7]:
import matplotlib.pyplot as plt

def plot_burndown(points: List[Dict[str, Any]], out_path: Path) -> str:
    if not points:
        return ""
    xs = [p["day"] for p in points]
    ys = [p["remaining_h"] for p in points]
    plt.figure(figsize=(7,4))
    plt.plot(xs, ys, marker="o")
    plt.title("Burndown (Remaining Hours)")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    out_path.parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(out_path)
    plt.close()
    return str(out_path)


In [8]:
import os, json
from google.adk.agents import LlmAgent
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types
from pathlib import Path

# model: your local llama.cpp alias
composer = LlmAgent(
    model=LiteLlm(model="openai/deepseek-local"),
    name="pm_status_reporter",
    description="Writes a concise project status report from tool outputs.",
    instruction=(
        "You are a project manager. Given JSON of tasks and a summary, produce a clear Markdown status report with:\n"
        "1) Summary KPIs (counts by status, remaining hours, ETA)\n"
        "2) Owner breakdown (bullets)\n"
        "3) Top risks/blockers (from any 'blocked' tasks)\n"
        "4) Upcoming deadlines (next 3 by due date)\n"
        "5) Next actions (bullets)\n"
        "Keep it to ~250–350 words. Do not apologize or refuse. Use only provided numbers."
    ),
    tools=[],
)

APP_NAME="adk_pm_demo"
USER_ID="student"
SESSION_ID="session-pm-001"

def build_pack() -> Dict[str, Any]:
    rows = _load()
    owners = sorted(set(r["owner"] for r in rows))
    summary = compute_summary()
    upcoming = sorted(rows, key=lambda r: r["due"])[:3]
    blocked = [r for r in rows if r["status"]=="blocked"]
    return {"owners": owners, "tasks": rows, "summary": summary, "upcoming": upcoming, "blocked": blocked}

async def generate_status_report() -> str:
    pack = build_pack()
    user_msg = (
        "Write a project status report from this JSON. Output Markdown only.\n\n"
        f"{json.dumps(pack, indent=2)}"
    )

    # try ADK; fallback if refusal/timeout
    session_service = InMemorySessionService()
    await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
    runner = Runner(agent=composer, app_name=APP_NAME, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=user_msg)])
    final_text = None; acc = ""

    try:
        for event in runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
            if getattr(event, "content", None) and getattr(event.content, "parts", None):
                t = getattr(event.content.parts[0], "text", None)
                if t:
                    acc += t; final_text = acc
                    if len(acc) > 1800: break
    except Exception as e:
        print("[LLM error] Fallback:", repr(e))
        final_text = None

    # simple fallback renderer
    bad = (not final_text) or any(k in final_text.lower() for k in ["i'm sorry","as an ai","cannot","unable"])
    if bad:
        s = pack["summary"]; counts = s.get("counts", {})
        lines = [
            "# Project Status Report",
            f"**KPI** — todo:{counts.get('todo',0)}, doing:{counts.get('doing',0)}, "
            f"blocked:{counts.get('blocked',0)}, done:{counts.get('done',0)}; "
            f"remaining: {int(s.get('remaining_h',0))}h; ETA: {s.get('eta_days','?')} days",
            "## Owner Breakdown",
        ]
        by_owner = {}
        for r in pack["tasks"]:
            by_owner.setdefault(r["owner"], []).append(r)
        for o, rows in by_owner.items():
            lines.append(f"- **{o}**: " + ", ".join(f"{r['title']}({r['status']})" for r in rows[:5]))
        if pack["blocked"]:
            lines.append("## Risks/Blockers")
            for r in pack["blocked"]:
                lines.append(f"- #{r['id']} {r['title']} — owner {r['owner']}, due {r['due']}")
        if pack["upcoming"]:
            lines.append("## Upcoming (next 3)")
            for r in pack["upcoming"]:
                lines.append(f"- #{r['id']} {r['title']} — {r['due']}")
        lines += ["## Next Actions", "- Unblock webhook", "- Finish auth flow", "- Prepare release notes"]
        final_text = "\n\n".join(lines)

    # save artifacts
    out_dir = Path("/content/agent_outputs"); out_dir.mkdir(parents=True, exist_ok=True)
    md_path = out_dir / f"pm_status_{datetime.now().strftime('%Y%m%d-%H%M%S')}.md"
    md_path.write_text(final_text)

    # chart
    bd = pack["summary"].get("burndown_points", [])
    chart_path = out_dir / f"pm_burndown_{datetime.now().strftime('%Y%m%d-%H%M%S')}.png"
    chart_file = plot_burndown(bd, chart_path) if bd else ""

    print("Saved:", md_path)
    if chart_file: print("Chart:", chart_file)
    return final_text

# Quick smoke test
print("Tasks now:", len(_load()))


Tasks now: 5


In [10]:
import os, requests

# Point ADK/LiteLLM to your local llama.cpp server
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:8000/v1"
os.environ["OPENAI_API_KEY"]  = "sk-local-anything"  # any string works for local

# Optional: shorter timeouts and fail-fast
os.environ["LITELLM_TIMEOUT"] = "120"
os.environ["LITELLM_REQUEST_TIMEOUT"] = "120"
os.environ["LITELLM_MAX_RETRIES"] = "0"

# Sanity probe (optional)
try:
    r = requests.get("http://127.0.0.1:8000/v1/models", timeout=5)
    print("Models:", r.json())
except Exception as e:
    print("Server not reachable yet:", e)


Models: {'object': 'list', 'data': [{'id': 'deepseek-local', 'object': 'model', 'owned_by': 'me', 'permissions': []}]}


In [11]:
# Create an extra task & update one, to show tool usage
_ = create_task("QA test plan","Nora", 4, (datetime.now().date()+timedelta(days=2)).isoformat(), ["qa"])
_ = update_status(3, "doing", spent_delta=1.0)  # unblock a task
report_md = await generate_status_report()
print(report_md[:1200])


Saved: /content/agent_outputs/pm_status_20251023-030550.md
Chart: /content/agent_outputs/pm_burndown_20251023-030550.png
# Project Status Report

**KPI** — todo:3, doing:3, blocked:0, done:1; remaining: 29h; ETA: 4.8 days

## Owner Breakdown

- **Aisha**: Auth flow(doing), Billing webhook(doing)

- **Miguel**: Landing page(todo)

- **Nora**: Onboarding email(done), QA test plan(todo), QA test plan(todo)

- **Ravi**: DB backup setup(doing)

## Upcoming (next 3)

- #4 Onboarding email — 2025-10-24

- #2 Landing page — 2025-10-25

- #6 QA test plan — 2025-10-25

## Next Actions

- Unblock webhook

- Finish auth flow

- Prepare release notes


In [12]:
# UI: Project Manager Agent (Gradio)
!pip -q install gradio nest_asyncio

import gradio as gr
import nest_asyncio, asyncio
from pathlib import Path
import pandas as pd

nest_asyncio.apply()

def _tasks_df():
    try:
        rows = list_tasks()["tasks"]
    except Exception:
        rows = []
    if not rows:
        return pd.DataFrame([{"info": "No tasks yet"}])
    df = pd.DataFrame(rows)
    # nicer column order
    cols = ["id","title","owner","status","estimate_h","spent_h","due","tags"]
    return df[[c for c in cols if c in df.columns]]

def ui_list_tasks():
    df = _tasks_df()
    return df

def ui_create_task(title, owner, estimate_h, due, tags_csv):
    tags = [t.strip() for t in (tags_csv or "").split(",") if t.strip()]
    res = create_task(title, owner, estimate_h, due, tags)
    t = res.get("created", {})
    msg = f"✅ Created task #{t.get('id')} — {t.get('title')} (owner: {t.get('owner')}, due: {t.get('due')})"
    return msg, _tasks_df()

def ui_update_status(task_id, status, spent_delta):
    try:
        res = update_status(int(task_id), status, float(spent_delta or 0))
    except Exception as e:
        return f"❌ Error: {e}", _tasks_df()
    if "updated" in res:
        u = res["updated"]
        msg = (f"✅ Updated #{u['id']} to {u['status']}; "
               f"spent_h now {u['spent_h']}")
    else:
        msg = "❌ Task not found."
    return msg, _tasks_df()

# run the async reporter safely from Gradio
def ui_generate_report():
    loop = asyncio.get_event_loop()
    if loop.is_running():
        # we are in a notebook; create a task and wait on it
        fut = asyncio.ensure_future(generate_status_report())
        loop.run_until_complete(asyncio.sleep(0))  # yield once
        loop.run_until_complete(fut)
        md = fut.result()
    else:
        md = loop.run_until_complete(generate_status_report())

    # find the latest files
    out_dir = Path("/content/agent_outputs")
    img_path = ""
    if out_dir.exists():
        pngs = sorted(out_dir.glob("pm_burndown_*.png"), reverse=True)
        if pngs:
            img_path = str(pngs[0])
    return md, img_path

with gr.Blocks(title="Project Manager Agent (ADK)") as demo:
    gr.Markdown("# Project Manager Agent (ADK)\nManage tasks and generate a status report with a local DeepSeek model.")

    with gr.Tab("Overview"):
        btn_refresh = gr.Button("🔄 Refresh")
        grid = gr.Dataframe(value=_tasks_df(), interactive=False, wrap=True)
        btn_refresh.click(fn=ui_list_tasks, outputs=grid)

    with gr.Tab("Create Task"):
        with gr.Row():
            title = gr.Textbox(label="Title", placeholder="Implement login form")
            owner = gr.Textbox(label="Owner", placeholder="Aisha")
        with gr.Row():
            estimate = gr.Number(label="Estimate (hours)", value=4, precision=1)
            due = gr.Textbox(label="Due Date (YYYY-MM-DD)", value=str(datetime.now().date()+timedelta(days=3)))
            tags = gr.Textbox(label="Tags (comma-separated)", value="frontend")
        btn_create = gr.Button("➕ Create")
        out_msg_create = gr.Markdown()
        out_grid_after_create = gr.Dataframe(interactive=False, wrap=True)
        btn_create.click(
            fn=ui_create_task,
            inputs=[title, owner, estimate, due, tags],
            outputs=[out_msg_create, out_grid_after_create]
        )

    with gr.Tab("Update Status"):
        with gr.Row():
            task_id = gr.Number(label="Task ID", precision=0, value=1)
            status = gr.Dropdown(choices=["todo","doing","blocked","done"], value="doing", label="New Status")
            spent_delta = gr.Number(label="Add Spent (hours)", value=1.0, precision=1)
        btn_update = gr.Button("✅ Update")
        out_msg_update = gr.Markdown()
        out_grid_after_update = gr.Dataframe(interactive=False, wrap=True)
        btn_update.click(
            fn=ui_update_status,
            inputs=[task_id, status, spent_delta],
            outputs=[out_msg_update, out_grid_after_update]
        )

    with gr.Tab("Generate Report"):
        gr.Markdown("Creates a Markdown status report and a burndown chart image in `/content/agent_outputs`.")
        btn_report = gr.Button("📝 Generate Status Report")
        report_md = gr.Markdown()
        chart_img = gr.Image(label="Burndown", show_label=True)
        btn_report.click(fn=ui_generate_report, outputs=[report_md, chart_img])

demo.launch(share=False)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>